In [56]:
from lib.utils.functions import *

In [57]:
characters_df = import_character_list('Got Characters.csv')

In [58]:
#remove the parentesis from character names in the df 
characters_df['name'] = characters_df['name'].apply(lambda x: re.sub('[\(.*?)]', '', x))
#creating a column for the first name of the characters
characters_df['first_name'] = characters_df['name'].apply(lambda x: x.split(' ', 1)[0])

In [59]:
characters_df

,Unnamed: 0,name,first_name
0,0,Addam Marbrand,Addam
1,1,Addison Hill,Addison
2,2,Adrack Humble,Adrack
3,3,Adrian Tarbeck,Adrian
4,4,Aegon Blackfyre,Aegon
...,...,...,...
1261,1261,Yorko Terys,Yorko
1262,1262,Ysabel,Ysabel
1263,1263,Ysilla,Ysilla
1264,1264,Ysilla Royce,Ysilla


In [60]:
all_books = [b for b in os.scandir(r"C:\Users\Thyfus\Desktop\dataProject\GoT project\Books") if '.txt' in b.name]

In [61]:
all_books

[<DirEntry '1 A Game of Thrones.txt'>,
 <DirEntry '2 A Clash of Kings.txt'>,
 <DirEntry '3 A Storm of Swords.txt'>,
 <DirEntry '4 A Feast for Crows.txt'>,
 <DirEntry '5 A Dance with Dragons.txt'>]

In [62]:
for book in all_books:
    
    #load the english module; NER stands for named entity recognition
    book_doc = NER(book)
    
    #loop through sentences, store named entities for each sentence
    sent_entity_df = get_ne_list_per_sentence(book_doc)
    
    #applying the function 
    sent_entity_df['character_entities'] = sent_entity_df['entities'].apply(lambda x: filter_entity(x, characters_df))
    
    #filtering out sentences without entities
    sent_entity_df_filtered = sent_entity_df[sent_entity_df['character_entities'].map(len) > 0]
    
    #Take only first name of characters
    sent_entity_df_filtered['character_entities'] = sent_entity_df_filtered['character_entities'].apply(lambda x: [item.split()[0] for item in x])
    
    #looking for relationships between characters by scanning the text and aggregating all the duplicated relationships, this will include reverser relationships; Eddard Stark	Bran and Bran	Eddard Stark for example
    relationships = create_relationships(df = sent_entity_df_filtered, window_size = 5)
    
    #creating the visualization of the relationship_df
    #this is a simplistic model so the only edge attribute to take into account will be the value column of the data frame
    relationships_graph(relationships)

TypeError: object of type 'nt.DirEntry' has no len()